In [ ]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
!wget -q https://dlcdn.apache.org/spark/spark-3.4.1/spark-3.4.1-bin-hadoop3.tgz
!tar xf spark-3.4.1-bin-hadoop3.tgz
!pip install -q findspark

In [2]:
import os
import findspark
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.1-bin-hadoop3"

findspark.init()

In [3]:
from google.colab import drive
drive.mount("/content/gdrive")

Mounted at /content/gdrive


In [4]:
from pyspark import SparkConf
from pyspark.sql import *
from pyspark.sql.functions import *
from pyspark.sql.types import *

config = SparkConf().setMaster('local').setAppName('lab53')
spark = SparkSession.builder.config(conf=config).getOrCreate()
sc = spark.sparkContext

DATASET_PATH = '/content/gdrive/MyDrive/invoices.csv'

In [5]:
invoice_df = spark.read \
    .format("csv") \
    .option("header", "true") \
    .option("inferSchema", "true") \
    .load(DATASET_PATH)

invoice_df.show()

+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|InvoiceNo|StockCode|         Description|Quantity|    InvoiceDate|UnitPrice|CustomerID|       Country|
+---------+---------+--------------------+--------+---------------+---------+----------+--------------+
|   536365|     null|WHITE HANGING HEA...|       6|01-12-2010 8.26|     2.55|     17850|United Kingdom|
|   536365|    71053| WHITE METAL LANTERN|       6|01-12-2010 8.26|     3.39|     17850|United Kingdom|
|   536365|   84406B|CREAM CUPID HEART...|       8|01-12-2010 8.26|     2.75|     17850|United Kingdom|
|   536365|   84029G|KNITTED UNION FLA...|       6|01-12-2010 8.26|     3.39|     17850|United Kingdom|
|   536365|   84029E|RED WOOLLY HOTTIE...|       6|01-12-2010 8.26|     3.39|     17850|United Kingdom|
|   536365|    22752|SET 7 BABUSHKA NE...|       2|01-12-2010 8.26|     7.65|     17850|United Kingdom|
|   536365|    21730|GLASS STAR FROSTE...|       6|01-12-2010 8.

Hãy sử dụng các thao tác Aggregration để tạo một Dataframe mới với cấu trúc như sau:
*   Count: Số lượng các dòng trong dataset.
*   TotalQuantity: Tổng của tất cả các trường Quantity trong dataset.
*   AvgPrice: Trung bình cộng của tất cả các trường UnitPrice trong dataset.
*   CountDistinct: Số lượng giá trị riêng biệt của trường InvoiceNo.

In [6]:
from ast import alias
from pyspark.sql import functions as f
invoice_df.select(f.count('*').alias('Count'),
                  f.sum('Quantity').alias('TotalQuantity'),
                  f.avg('UnitPrice').alias('AvgPrice'),
                  f.countDistinct('InvoiceNo').alias('CountDistinct')
                  ).show()

+------+-------------+------------------+-------------+
| Count|TotalQuantity|          AvgPrice|CountDistinct|
+------+-------------+------------------+-------------+
|541909|      5176450|4.6111136260884855|        25900|
+------+-------------+------------------+-------------+



Hãy sử dụng các thao tác Aggregration để tạo một Dataframe mới với cấu trúc như sau:
*   Count: Số lượng các dòng trong dataset.
*   TotalQuantity: Tổng của tất cả các trường Quantity trong dataset.
*   AvgPrice: Trung bình cộng của tất cả các trường UnitPrice trong dataset.

Ở phần này, bạn hãy sử dụng dưới dạng String Expression

In [14]:
invoice_df.selectExpr(
     "count(*) as `count 1`",
     "count(StockCode) as `count field`",
     "sum(Quantity) as TotalQuantity",
     "avg(UnitPrice) as AvgPrice"
     ).show()

+-------+-----------+-------------+-----------------+
|count 1|count field|TotalQuantity|         AvgPrice|
+-------+-----------+-------------+-----------------+
| 541909|     541908|      5176450|4.611113626083471|
+-------+-----------+-------------+-----------------+



Hãy sử dụng các thao tác Aggregration để tạo một Dataframe mới với cấu trúc như sau, dữ liệu trong Dataframe này sẽ được gộp nhóm theo hai trường Country và InvoiceNo:
*   Country: Tên đất nước.
*   InvoiceNo: Mã số hóa đơn.
*   TotalQuantity: Tổng của tất cả các trường Quantity.
*   InvoiceValue: Tổng giá trị của hóa đơn, mỗi hóa đơn sẽ có Value = Quantity * UnitPrice. Hãy làm tròn giá trị này thành 2 chữ số thập phân.

Ở phần này, bạn hãy sử dụng dưới dạng SQL

In [21]:
invoice_df.createOrReplaceTempView('sales')
summary_sql = spark.sql("""
    SELECT df.Country, df.InvoiceNo, sum(df.Quantity) as TotalQuantity, round(sum(df.Quantity * df.UnitPrice)) as InvoiceValue
    FROM sales AS df
    GROUP BY df.Country, df.InvoiceNo
                        """
    )
summary_sql.show()

+--------------+---------+-------------+------------+
|       Country|InvoiceNo|TotalQuantity|InvoiceValue|
+--------------+---------+-------------+------------+
|United Kingdom|   536446|          329|       441.0|
|United Kingdom|   536508|          216|       156.0|
|United Kingdom|   537018|           -3|         0.0|
|United Kingdom|   537401|          -24|         0.0|
|United Kingdom|   537811|           74|       269.0|
|United Kingdom|  C537824|           -2|       -15.0|
|United Kingdom|   538895|          370|       247.0|
|United Kingdom|   540453|          341|       302.0|
|United Kingdom|   541291|          217|       306.0|
|United Kingdom|   542551|           -1|         0.0|
|United Kingdom|   542576|           -1|         0.0|
|United Kingdom|   542628|            9|       132.0|
|United Kingdom|   542886|          199|       321.0|
|United Kingdom|   542907|           75|       314.0|
|United Kingdom|   543131|          134|       164.0|
|United Kingdom|   543189|  

Tương tự với đề bài bên trên, nhưng bạn hãy sử dụng Object Expression để thực hiện.

In [23]:
summary_df = invoice_df \
    .groupBy("Country", "InvoiceNo") \
    .agg(f.sum("Quantity").alias("TotalQuantity"),
         f.round(f.sum(expr('Quantity * UnitPrice'))).alias("InvoiceValue"))
summary_df.show()

+--------------+---------+-------------+------------+
|       Country|InvoiceNo|TotalQuantity|InvoiceValue|
+--------------+---------+-------------+------------+
|United Kingdom|   536446|          329|       441.0|
|United Kingdom|   536508|          216|       156.0|
|United Kingdom|   537018|           -3|         0.0|
|United Kingdom|   537401|          -24|         0.0|
|United Kingdom|   537811|           74|       269.0|
|United Kingdom|  C537824|           -2|       -15.0|
|United Kingdom|   538895|          370|       247.0|
|United Kingdom|   540453|          341|       302.0|
|United Kingdom|   541291|          217|       306.0|
|United Kingdom|   542551|           -1|         0.0|
|United Kingdom|   542576|           -1|         0.0|
|United Kingdom|   542628|            9|       132.0|
|United Kingdom|   542886|          199|       321.0|
|United Kingdom|   542907|           75|       314.0|
|United Kingdom|   543131|          134|       164.0|
|United Kingdom|   543189|  